<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

# BirdCLEF 2024 with [KerasCV](https://github.com/keras-team/keras-cv) and [Keras](https://github.com/keras-team/keras)

> The objective of this competition is to identify under-studied Indian bird species by their calls.

<div align="center">
  <img src="https://i.ibb.co/47F4P9R/birdclef2024.png">
</div>

This notebook guides you through the process of inferring a Deep Learning model to recognize bird species by their songs (audio data). As the inference requires running only on the `CPU`, we had to create a separate notebooks for training and inference. You can find the [training notebook here](https://www.kaggle.com/code/awsaf49/birdclef24-kerascv-starter-train). Just as a recap of the training notebook, it uses the EfficientNetV2 backbone from KerasCV on the competition dataset. That notebook also demonstrates how to convert audio data to mel-spectrograms using Keras.

<u>Fun fact</u>: Both the training and inference notebooks are backend-agnostic, supporting TensorFlow, PyTorch, and JAX. Utilizing KerasCV and Keras allows us to choose our preferred backend. Explore more details on [Keras](https://keras.io/keras_core/announcement/).

In this notebook, you will learn:

- Designing a data pipeline for audio data, including audio-to-spectrogram conversion.
- Loading the data efficiently using [`tf.data`](https://www.tensorflow.org/guide/data).
- Creating the model using KerasCV presets.
- Inferring the trained model.

**Note**: For a more in-depth understanding of KerasCV, refer to the [KerasCV guides](https://keras.io/guides/keras_cv/).

# Import Libraries 📚

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

2024-04-20 13:08:05.974454: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 13:08:05.974597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 13:08:06.139994: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/1585041062.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


# Configuration ⚙️

In [2]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    preset = 'efficientnetv2_b2_imagenet'

    # Class Labels for BirdCLEF 24
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

# Reproducibility ♻️
Sets value for random seed to produce similar result in each run.

In [3]:
tf.keras.utils.set_random_seed(CFG.seed)

# Dataset Path 📁

In [4]:
BASE_PATH = '/kaggle/input/birdclef-2024'

# Test Data 📖

In [5]:
test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `unlabeled` data as there is no `test` data.
# During submission `test` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,/kaggle/input/birdclef-2024/unlabeled_soundsca...
1,/kaggle/input/birdclef-2024/unlabeled_soundsca...
2,/kaggle/input/birdclef-2024/unlabeled_soundsca...
3,/kaggle/input/birdclef-2024/unlabeled_soundsca...
4,/kaggle/input/birdclef-2024/unlabeled_soundsca...


# Modeling 🤖

Note that our model was trained on `10 second` duration audio files, but we will infer on `5-second` audio files (as per competition rules). To facilitate this, we have set the model input shape to `(None, None, 3)`, which will allow us to have variable-length input during training and inference.

In [6]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    CFG.preset,
)
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = keras.models.Model(inputs=inp, outputs=out)
# Load weights of trained model
model.load_weights("/kaggle/input/birdclef24-kerascv-starter-train/best_model.weights.h5")

Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...


# Data Loader 🍚

The following code will decode the raw audio from `.ogg` file and also decode the spectrogram from the `audio` file. Additionally, we will apply Z-Score standardization and Min-Max normalization to ensure consistent inputs to the model.

In [7]:
# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1, 1, 1, 3])
        return spec
    
    return decode

In [8]:
# Build data loader
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

# Inference 🏃

In [9]:
# Initialize empty list to store ids
ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)

test : 100%|██████████| 10/10 [00:23<00:00,  2.37s/it]


# Submission ✉️

In [10]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('submission.csv',index=False)
pred_df.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.004292,0.005831,0.000661,0.000982,0.002183,0.000507,0.002637,0.000673,0.000792,...,0.002693,0.000534,0.001849,0.005823,0.008234,0.002441,0.000124,0.000262,0.000440,0.001411
1,1872382287_10,0.003028,0.004529,0.000466,0.000658,0.003599,0.000515,0.003603,0.001392,0.000694,...,0.004691,0.000864,0.004394,0.004521,0.006326,0.003542,0.000252,0.000389,0.000559,0.001547
2,1872382287_15,0.002689,0.006433,0.000501,0.000498,0.003042,0.000261,0.003481,0.001180,0.001625,...,0.005297,0.000624,0.001575,0.004882,0.003599,0.001388,0.000271,0.000254,0.000853,0.000525
3,1872382287_20,0.003968,0.005301,0.000409,0.000337,0.002578,0.000546,0.001688,0.001314,0.001499,...,0.003899,0.000512,0.001381,0.003995,0.006398,0.002049,0.000157,0.000545,0.000509,0.001253
4,1872382287_25,0.001835,0.001738,0.000651,0.000423,0.006607,0.000982,0.001143,0.001365,0.001177,...,0.006686,0.000833,0.003624,0.002203,0.003343,0.007066,0.000207,0.000513,0.000622,0.001363


# Reference ✍️
* [Fake Speech Detection: Conformer [TF]](https://www.kaggle.com/code/awsaf49/fake-speech-detection-conformer-tf/) by @awsaf49
* [RANZCR: EfficientNet TPU Training](https://www.kaggle.com/code/xhlulu/ranzcr-efficientnet-tpu-training) by @xhlulu
* [Triple Stratified KFold with TFRecords](https://www.kaggle.com/code/cdeotte/triple-stratified-kfold-with-tfrecords) by @cdeotte